In [1]:
import numpy as np

In [48]:
ts1 = np.loadtxt("ts1.txt")
ts2 = np.loadtxt("ts2.txt")
ts3 = np.loadtxt("ts3.txt")

In [4]:
from Operations.ST_LocalExtrema import ST_LocalExrema as STLE

In [1]:
from ctypes import *
import ctypes

In [7]:
so_file = "/Users/joshua/Desktop/MS_shannon.so"
lib = CDLL(so_file)

In [8]:
lib.entropy.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, flags='C_CONTIGUOUS'),
    ctypes.c_int,
    ctypes.c_int,
    ctypes.c_int,
    ctypes.POINTER(ctypes.c_double)
]
lib.entropy.restype = None  # void function

In [9]:
def shannon_entropy(data, bin_count, depth):
    """
    Calculate Shannon Entropy using the C function.
    
    Args:
    data (numpy.array): Input time series data
    bin_count (int): Number of bins for encoding
    depth (int): Depth of the encoding
    
    Returns:
    float: Calculated Shannon Entropy
    """
    data = np.ascontiguousarray(data, dtype=np.float64)
    length = len(data)
    result = ctypes.c_double()
    
    lib.entropy(data, length, bin_count, depth, ctypes.byref(result))
    
    return result.value

In [10]:
shannon_entropy(ts3, 2, 5)

3.258990526199341

In [12]:
numBins = 10
numBins = [numBins]

In [82]:
import numpy as np
from Operations.CO_HistogramAMI import CO_HistogramAMI
from scipy import stats
from PeripheryFunctions.BF_SignChange import BF_SignChange

def CO_CompareMinAMI(y, binMethod, numBins = 10):
    """
    """
    N = len(y)
    # Range of time lags to consider
    tauRange = np.arange(0, int(np.ceil(N/2))+1)
    numTaus = len(tauRange)

    # range of bin numbers to consider
    if isinstance(numBins, int):
        numBins = [numBins]
    
    numBinsRange = len(numBins)
    amiMins = np.zeros(numBinsRange)

    # Calculate automutual information
    for i in range(numBinsRange):  # vary over number of bins in histogram
        amis = np.zeros(numTaus)
        for j in range(numTaus):  # vary over time lags, tau
            amis[j] = CO_HistogramAMI(y, tauRange[j], binMethod, numBins[i])
            if (j > 1) and ((amis[j] - amis[j-1]) * (amis[j-1] - amis[j-2]) < 0):
                amiMins[i] = tauRange[j-1]
                break
        if amiMins[i] == 0:
            amiMins[i] = tauRange[-1]
    # basic statistics
    out = {}
    out['min'] = np.min(amiMins)
    out['max'] = np.max(amiMins)
    out['range'] = np.ptp(amiMins)
    out['median'] = np.median(amiMins)
    out['mean'] = np.mean(amiMins)
    out['std'] = np.std(amiMins, ddof=1)
    out['nunique'] = len(np.unique(amiMins))
    out['mode'], out['modef'] = stats.mode(amiMins)
    out['modef'] = out['modef']/numBinsRange

    # converged value? 
    out['conv4'] = np.mean(amiMins[-5:])

    # look for peaks (local maxima)
    # % local maxima above 1*std from mean
    # inspired by curious result of periodic maxima for periodic signal with
    # bin size... ('quantiles', [2:80])
    diff_ami_mins = np.diff(amiMins[:-1])
    positive_diff_indices = np.where(diff_ami_mins > 0)[0]
    sign_change_indices = BF_SignChange(diff_ami_mins, 1)

    # Find the intersection of positive_diff_indices and sign_change_indices
    loc_extr = np.intersect1d(positive_diff_indices, sign_change_indices) + 1
    above_threshold_indices = np.where(amiMins > out['mean'] + out['std'])[0]
    big_loc_extr = np.intersect1d(above_threshold_indices, loc_extr)

    # Count the number of elements in big_loc_extr
    out['nlocmax'] = len(big_loc_extr)

    return out


In [85]:
CO_CompareMinAMI(ts1, 'even', range(2,81))

{'min': 2.0,
 'max': 9.0,
 'range': 7.0,
 'median': 6.0,
 'mean': 5.620253164556962,
 'std': 1.7708740234361344,
 'nunique': 8,
 'mode': 4.0,
 'modef': 0.189873417721519,
 'conv4': 6.6,
 'nlocmax': 7}

In [7]:
from Operations.CO_HistogramAMI import CO_HistogramAMI

In [20]:
import numpy as np
from Operations.CO_FirstCrossing import CO_FirstCrossing

def CO_HistogramAMI3(y, tau = 1, meth = 'even', numBins = 10):
    """
    CO_HistogramAMI: The automutual information of the distribution using histograms.

    Parameters:
    y (array-like): The input time series
    tau (int, list or str): The time-lag(s) (default: 1)
    meth (str): The method of computing automutual information:
                'even': evenly-spaced bins through the range of the time series,
                'std1', 'std2': bins that extend only up to a multiple of the
                                standard deviation from the mean of the time series to exclude outliers,
                'quantiles': equiprobable bins chosen using quantiles.
    num_bins (int): The number of bins (default: 10)

    Returns:
    float or dict: The automutual information calculated in this way.
    """
    # Use first zero crossing of the ACF as the time lag
    if isinstance(tau, str) and tau in ['ac', 'tau']:
        tau = CO_FirstCrossing(y, 'ac', 0, 'discrete')
    
    # Bins for the data
    # same for both -- assume same distribution (true for stationary processes, or small lags)
    if meth == 'even':
        b = np.linspace(np.min(y), np.max(y), numBins + 1)
        # Add increment buffer to ensure all points are included
        inc = 0.1
        b[0] -= inc
        b[-1] += inc
    elif meth == 'std1': # bins out to +/- 1 std
        b = np.linspace(-1, 1, numBins + 1)
        if np.min(y) < -1:
            b = np.concatenate(([np.min(y) - 0.1], b))
        if np.max(y) > 1:
            b = np.concatenate((b, [np.max(y) + 0.1]))
    elif meth == 'std2': # bins out to +/- 2 std
        b = np.linspace(-2, 2, numBins + 1)
        if np.min(y) < -2:
            b = np.concatenate(([np.min(y) - 0.1], b))
        if np.max(y) > 2:
            b = np.concatenate((b, [np.max(y) + 0.1]))
    elif meth == 'quantiles': # use quantiles with ~equal number in each bin
        b = np.quantile(y, np.linspace(0, 1, numBins + 1))
        b[0] -= 0.1
        b[-1] += 0.1
    else:
        raise ValueError(f"Unknown method '{meth}'")
    
    # Sometimes bins can be added (e.g., with std1 and std2), so need to redefine numBins
    numBins = len(b) - 1

    # Form the time-delay vectors y1 and y2
    if not isinstance(tau, (list, np.ndarray)):
        # if only single time delay as integer, make into a one element list
        tau = [tau]

    amis = np.zeros(len(tau))

    for i, t in enumerate(tau):
        if t == 0:
            # for tau = 0, y1 and y2 are identical to y
            y1 = y2 = y
        else:
            y1 = y[:-t]
            y2 = y[t:]
        # Joint distribution of y1 and y2
        pij, _, _ = np.histogram2d(y1, y2, bins=(b, b))
        pij = pij[:numBins, :numBins]  # joint
        pij = pij / np.sum(pij)  # normalize
        pi = np.sum(pij, axis=1)  # marginal
        pj = np.sum(pij, axis=0)  # other marginal

        pii = np.tile(pi, (numBins, 1)).T
        pjj = np.tile(pj, (numBins, 1))

        r = pij > 0  # Defining the range in this way, we set log(0) = 0
        amis[i] = np.sum(pij[r] * np.log(pij[r] / pii[r] / pjj[r]))

    if len(tau) == 1:
        return amis[0]
    else:
        return {f'ami{i+1}': ami for i, ami in enumerate(amis)}
